In [2]:
import pandas as pd
# pd.set_option('display.max_columns', None)
# pd.set_option('display.width', 1000)
# pd.set_option('max_colwidth', 2000)
import MySQLdb
import numpy as np

def retrieve_to_frame(
    sql, 
    host="",
    user="",
    passwd="=",
    db="",
    charset='utf8'
):
    """
    """
    conn = MySQLdb.connect(host=host, user=user, passwd=passwd,
                           db=db, charset=charset)
    sql = sql
    result_df = pd.read_sql(sql, con=conn)
    conn.close()
    return result_df

In [3]:
# df=retrieve_to_frame(f'SHOW FULL FIELDS FROM db.tbl')[['Field','Comment']]
# # df[df['Field'].isin([item for item in df['Field'] if 'money'  in item])]
# df

In [16]:
def add_indent(single_line,indent_level=0,indent_base=' '*4):
    return indent_base*indent_level+single_line

def construct_query_function_header(query_function_name='fun',table_alias='tbl',indent_level=0):
    function_string = []
    function_string.append(add_indent(single_line=f"def {query_function_name}_sql(",indent_level=indent_level))
    indent_level += 1
    param_strings = [
        "###########",
        f"# {table_alias}",
        "# ----------------------",
        "str_param: str = '',",
        "params: tuple = (),",
        "column_list: tuple = ('',),",
        "select_all: bool = False,",
        "groupby_columns: tuple = ('',),",
        "more_where_conditions: dict = {'normal':()}"
    ]
    for single_line in param_strings:
        function_string.append(add_indent(single_line=single_line,indent_level=indent_level))
    indent_level -= 1
    function_string.append(add_indent(single_line="):",indent_level=indent_level))
    return print('\n'.join(function_string))

In [21]:
def construct_mysql_query_function_in_python(database_name='db',table_name='table',table_alias='tbl',query_function_name='func'):
    construct_query_function_header(query_function_name=query_function_name,table_alias=table_alias,indent_level=0)
    
    df=retrieve_to_frame(f'SHOW FULL FIELDS FROM {database_name}.{table_name}')[['Field','Comment']]
    column_entry_dict = {}
    for index in df.index:
        column=df.iloc[index,0]
        column_entry_dict[column] = {
            'column_expression':f'''{table_alias}.{column}''',
            'column_alias':column,
            'comment':df.iloc[index,1]
        }
    function_string = ['\n']
    indent_level=1
    
    strings = [
        "# "+str(len(df)),
        "column_entry_dict = {",
    ]
    for key,value in column_entry_dict.items():
        strings.append('# '+value.pop('comment'))
        strings.append(repr(key)+' : """\n'+\
                       add_indent(single_line=value['column_expression'],indent_level=indent_level+1)+\
                       '\n\n'+add_indent(single_line='""",\n',indent_level=indent_level+1))
    
    strings.append(add_indent(single_line="}",indent_level=indent_level))
        
    for single_line in strings:
        function_string.append(add_indent(single_line=single_line,indent_level=indent_level))
    print('\n'.join(function_string))
    function_string=[]
    
    strings = [
        "sql_strings = ['"+add_indent(single_line='SELECT',indent_level=indent_level)+"']",
        "if select_all:",
        add_indent(single_line="for key,value in column_entry_dict.items():",indent_level=indent_level),
        add_indent(single_line="sql_strings.append(f'''"+' '*8+"{value.strip()}    {key},''')",indent_level=indent_level+1),
        "else:",
        add_indent(single_line="for item in column_list:",indent_level=indent_level),
        add_indent(single_line="temp = column_entry_dict.get(item['column_key'])",indent_level=indent_level+1),
        add_indent(single_line="if temp is None:",indent_level=indent_level+1),
        add_indent(single_line="import warnings",indent_level=indent_level+2),
        add_indent(single_line="warnings.warn('   invalid columns keys：'+item['column_key'])",indent_level=indent_level+2),
        
        add_indent(single_line="else:",indent_level=indent_level+1),
        add_indent(single_line="column_expression =item['column_expression'] if item['column_expression'] else temp",indent_level=indent_level+2),
        add_indent(single_line="column_alias =item['column_alias'] if item['column_alias'] else item['column_key']",indent_level=indent_level+2),
        add_indent(single_line="sql_strings.append(f'''{column_expression.rstrip()}   {column_alias.rstrip()},''')",indent_level=indent_level+2),
        '\n',
        "sql_strings[-1] = sql_strings[-1].rstrip()[:-1]",
        "sql_strings.append('''",
        f"FROM {database_name}.{table_name} AS {table_alias}",
        "WHERE 1=1",
        "''')",
        
        "###########",
        f"# {table_alias}",
        "# ----------------------",
        "if params:",
        add_indent(single_line="if len(params) == 1:",indent_level=indent_level),
        add_indent(single_line="params *= 2",indent_level=indent_level+1),
        add_indent(single_line=f"""sql_strings.append(f"        AND {table_alias}.param IN """+ """{params}") """,indent_level=indent_level),
        "else:",
        add_indent(single_line="raise NotImplementedError('请提供 params!')",indent_level=indent_level),
        '\n',
        "if str_param:",
        add_indent(single_line=f""" sql_strings.append(f"        AND {table_alias}.str_param"""+""" < '{str_param}'") """,indent_level=indent_level),
        "else:",
        add_indent(single_line="raise NotImplementedError('请提供 str_param!')",indent_level=indent_level),
        '\n',
        "if more_where_conditions:",
        add_indent(single_line="if more_where_conditions.get('normal'):",indent_level=indent_level),
        add_indent(single_line="sql_strings.extend(list(more_where_conditions['normal']))",indent_level=indent_level+1),
        '\n',
        "return '\\n'.join(sql_strings)"
    ]
    
    for single_line in strings:
        function_string.append(add_indent(single_line=single_line,indent_level=indent_level))    
    print('\n'.join(function_string))

In [ ]:
goods_info_base_price_ufen_v2=retrieve_to_frame(
    goods_info_base_price_sql_ufen_v2(
        select_all=False,
        supplier_codes = ('50', '100', '0737000000000000'),
        column_list=(
            {'column_key':'category_code','column_expression':'','column_alias':''},
            {'column_key':'supplier_code','column_expression':'','column_alias':''},
            {'column_key':'SKU_code','column_expression':'','column_alias':''},
            {'column_key':'SKU_code_ufen_v1','column_expression':'','column_alias':''},
            {'column_key':'SKU名称','column_expression':'','column_alias':''},
            {'column_key':'品牌','column_expression':'','column_alias':''},
            {'column_key':'SKU是否可售','column_expression':'','column_alias':''},
            {'column_key':'SKU创建时间','column_expression':'','column_alias':''},
            {'column_key':'SKU销售单价','column_expression':'','column_alias':''},
            {'column_key':'SKU采购价','column_expression':'','column_alias':''},
        ),
        more_where_conditions={'normal':("AND (gds_info_bs.brand like '%三只松鼠%' OR gds_info_bs.goods_name like '%三只松鼠%' ) ",)}
    )
)

In [4]:
# construct_mysql_query_function_in_python(database_name='database_name',table_name='table_name',table_alias='table_alias',query_function_name='query_function_name')

In [5]:
# print(goods_info_price_sql(str_param='--',params=('--',),select_all=True))